In [1]:
from torch import cuda, bfloat16
import transformers

model_name = '/home/ubuntu/model/falcon-40b-instruct'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto'
)
model.eval()
print(f"Model loaded on {device}")

/home/ubuntu/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/dev/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /home/ubuntu/dev/bin/python: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizers and quantization routines!!
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 00
CUDA SETUP: Loading bi

/home/ubuntu/dev/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('https'), PosixPath('//debuginfod.ubuntu.com ')}
  warn(msg)
/home/ubuntu/dev/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/ubuntu/dev/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/ubuntu/dev/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg

Model loaded on cuda:0


In [2]:
import transformers
from transformers import StoppingCriteria, StoppingCriteriaList
import torch


tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
# we create a list of stopping criteria
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [
        ['Human', ':'], ['AI', ':']
    ]
]
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

In [3]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [4]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'R

In [5]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)

In [43]:
# Import os to set API key
import os
# Import OpenAI as main LLM service
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
# Bring in streamlit for UI/app interface
# Import PDF document loaders...there's other ones as well!
from langchain.document_loaders import PyPDFLoader
# Import chroma as the vector store 
from langchain.vectorstores import Chroma

# Import vector store stuff
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo
)

from dotenv import dotenv_values
env_variables = dotenv_values('env.env')
# Set APIkey for OpenAI Service
# Can sub this out for other LLM providers
os.environ['OPENAI_API_KEY'] = env_variables['Open_API']

In [44]:
embeddings = OpenAIEmbeddings()

In [45]:
text = "hello word"
embed = embeddings.embed_query(text)
print(len(embed))

1536


In [46]:
# Create and load PDF Loader
loader = PyPDFLoader('paper.pdf')
# Split pages from pdf 
pages = loader.load_and_split()[0:1]

### make a directory called chroma_db

In [1]:
# load documents into vector database aka ChromaDB and store in on the disk
db5 = Chroma.from_documents(pages, embeddings, persist_directory="./chroma_db")
db5.persist()


NameError: name 'Chroma' is not defined

In [ ]:
# send a query to DB based on the similarity between embedding of query and data in DB
query = "What is the title of paper"
db5.similarity_search(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


[Document(page_content='Solar Irradiance Anticipative Transformer\nThomas M. Mercier\nBournemouth University\ntmercier2@gmail.comTasmiat Rahman\nUniversity of Southampton\nt.rahman@soton.ac.ukAmin Sabet\nEscherCloud AI\na.sabet@eschercloud.ai\nAbstract\nThis paper proposes an anticipative transformer-based\nmodel for short-term solar irradiance forecasting. Given a\nsequence of sky images, our proposed vision transformer\nencodes features of consecutive images, feeding into a\ntransformer decoder to predict irradiance values associated\nwith future unseen sky images. We show that our model ef-\nfectively learns to attend only to relevant features in im-\nages in order to forecast irradiance. Moreover, the pro-\nposed anticipative transformer captures long-range depen-\ndencies between sky images to achieve a forecasting skill of\n21.45 % on a 15 minute ahead prediction for a newly intro-\nduced dataset of all-sky images when compared to a smart\npersistence model.\n1. Introduction\nSol

In [ ]:
# Load DB stored on the disk and send a query to it
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
docs = db3.similarity_search(query)
print(docs[0].page_content)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Solar Irradiance Anticipative Transformer
Thomas M. Mercier
Bournemouth University
tmercier2@gmail.comTasmiat Rahman
University of Southampton
t.rahman@soton.ac.ukAmin Sabet
EscherCloud AI
a.sabet@eschercloud.ai
Abstract
This paper proposes an anticipative transformer-based
model for short-term solar irradiance forecasting. Given a
sequence of sky images, our proposed vision transformer
encodes features of consecutive images, feeding into a
transformer decoder to predict irradiance values associated
with future unseen sky images. We show that our model ef-
fectively learns to attend only to relevant features in im-
ages in order to forecast irradiance. Moreover, the pro-
posed anticipative transformer captures long-range depen-
dencies between sky images to achieve a forecasting skill of
21.45 % on a 15 minute ahead prediction for a newly intro-
duced dataset of all-sky images when compared to a smart
persistence model.
1. Introduction
Solar energy has emerged as one of the most promis

In [ ]:
vectorstore_info = VectorStoreInfo(
    name="paper",
    description="a scietfic paper as a pdf",
    vectorstore=db3
)


In [ ]:
# Convert the document store into a langchain toolkit
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info,llm = llm)

In [ ]:
agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [ ]:
# send a query to llm to find the answer using info in DB
agent_executor.run("what is the title of this paper?")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new AgentExecutor chain...
 I need to find the title of the paper
Action: paper
Action Input: the name of the paper

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Observation:  Solar Irradiance Anticipative Transformer
Thought: I now know the title of the paper
Final Answer: The title of the paper is "Solar Irradiance Anticipative Transformer".

> Finished chain.


'The title of the paper is "Solar Irradiance Anticipative Transformer".'